# max_div_seq Runner

In [17]:
SEMESTER = '2017B'

In [18]:
import pybryt
import os
from shutil import copy, rmtree
from tqdm import tqdm

In [19]:
ref = pybryt.ReferenceImplementation.compile("max_even_seq.ipynb")
ref.dump("max_even_seq-ref.pkl")
ref = pybryt.ReferenceImplementation.load("max_even_seq-ref.pkl")

In [20]:
from glob import glob
subs = glob(f'subs/{SEMESTER}/*.py')

In [21]:
import IPython.nbformat  as nbf
from pathlib import Path

notebooks = []

for sub in subs:

    code = Path(sub).read_text()

    cells = []

    cell = nbf.v4.new_code_cell(source=code)
    cells.append(cell)
    cell = nbf.v4.new_code_cell(source="max_even_seq(222)")
    cells.append(cell)

    nb = nbf.v4.new_notebook(cells = cells)

    pre, ext = os.path.splitext(sub)
    notebook = pre + '.ipynb'
    with open(notebook, 'w') as f:

            nbf.write(nb, f, 4)
    notebooks.append(notebook)

In [ ]:
student_impls = []
for sub in tqdm(notebooks):
    try:
        student_impls.append(pybryt.StudentImplementation(sub))
    except Exception as e:
        print(f'{type(e).__name__} in {sub}')
        student_impls.append(None)

  6%|▌         | 11/198 [00:17<04:48,  1.54s/it]

In [ ]:
results = []
for si in student_impls:
    if si is None:
        results.append(None)
    else:
        results.append(si.check(ref))

In [ ]:
path = f'./advices/{SEMESTER}'
if os.path.exists(path):
    rmtree(path)
os.makedirs(path)
for sp, res in zip(subs, results):
    if res is None:
        os.makedirs(f'{path}/Error', exist_ok=True)
        copy(sp, f'{path}/Error/')
        continue
    if res.correct:
        os.makedirs(f'{path}/None', exist_ok=True)
        copy(sp, f'{path}/None/')
        continue
    for message in res.messages:
        os.makedirs(f'{path}/{message}', exist_ok=True)
        copy(sp, f'{path}/{message}/')